In [1]:
import Data
import regression
import dclassifier
import numpy as np
import pandas as pd
import backtest
import Genetic_Algorithm_2
import matplotlib.pyplot as plt
import pickle

In [2]:
training_months = [pd.Period('2016-04', 'M'), pd.Period('2016-05', 'M'), pd.Period('2016-06', 'M'), pd.Period('2016-07', 'M')]

data = Data.Data()

prices = data.prices
prices = prices[prices.index.to_period('M').isin(training_months)]

2000-01      4
2000-02     10
2000-03     46
2000-04    191
2000-05    191
          ... 
2019-11      4
2019-12     36
2020-01     24
2020-02      0
2020-03     71
Freq: M, Length: 243, dtype: int64


In [3]:
thresh_USDJPY = [0.017, 0.023, 0.025, 0.027000000000000003, 0.014000000000000002] #0.026
thresh_EURCHF = [0.011, 0.012, 0.013000000000000001, 0.010000000000000002, 0.019000000000000003]
thresh_EURUSD = [0.022000000000000002, 0.021, 0.020, 0.014000000000000002, 0.018000000000000002]
thresh_EURJPY = [0.024, 0.025, 0.026000000000000002, 0.027000000000000003, 0.023]
thresh_USDCHF = [0.017, 0.018000000000000002, 0.026000000000000002, 0.029, 0.014000000000000002]
thresh_USDCAD = [0.018000000000000002, 0.023, 0.024, 0.021, 0.019000000000000003]
combinations_USDJPY = []
combinations_EURCHF = []
combinations_EURUSD = []
combinations_EURJPY = []
combinations_USDCHF = []
combinations_USDCAD = []

for j in thresh_USDJPY:
    combinations_USDJPY.append((j, 'USDJPY'))

for j in thresh_EURCHF:
    combinations_EURCHF.append((j, 'EURCHF'))

for j in thresh_EURUSD:
    combinations_EURUSD.append((j, 'EURUSD'))

for j in thresh_EURJPY:
    combinations_EURJPY.append((j, 'EURJPY'))

for j in thresh_USDCHF:
    combinations_USDCHF.append((j, 'USDCHF'))

for j in thresh_USDCAD:
    combinations_USDCAD.append((j, 'USDCAD'))

In [4]:
def print_node(node):
    string = ''
    if node.val:
        string += (node.val if isinstance(node.val, str) else str(round(node.val, 3)))
    if len(node.children):
        string += f" ({' , '.join([print_node(child) for child in node.children])})"
    return string

In [5]:
UJ = {}
with open('USDJPY.pkl', "rb") as f:
    for combination in combinations_USDJPY:
        thresh, val = combination
        UJ[thresh] = [pickle.load(f)]
        print(print_node(UJ[thresh][0]))
        UJ[thresh].append(pickle.load(f))
        print(print_node(UJ[thresh][1]))


/ (/ (- (-7.461 , -0.886) , cos (DCl)) , log (cos (DCl)))
+ (+ (/ (9.334 , 4.588) , DCl) , + (/ (DCl , DCl) , / (DCl , 9.334)))
/ (+ (- (DCl , DCl) , / (9.61 , -8.651)) , log (sin (DCl)))
+ (/ (cos (6.687) , -2.666) , * (sin (0.792) , + (-7.11 , DCl)))
/ (+ (sin (DCl) , * (7.817 , 3.711)) , sin (log (DCl)))
* (/ (- (DCl , 8.461) , * (2.278 , 8.869)) , / (+ (DCl , 9.848) , * (2.278 , 8.461)))
- (sin (+ (-0.397 , -0.397)) , * (6.155 , - (-3.336 , 5.716)))
/ (* (cos (-7.583) , + (DCl , 1.232)) , / (* (7.172 , -1.835) , / (DCl , -8.099)))
/ (+ (-7.292 , sin (-5.025)) , log (cos (DCl)))
- (- (+ (DCl , 7.116) , cos (DCl)) , + (* (DCl , 0.552) , cos (DCl)))


In [6]:
class e_handler:
    def __init__(self, classified_data, theta, regressor) -> None:
        #, regressor
        self.classified_data = classified_data
        self.dc_count = 0
        self.detector = None
        self.theta = theta
        self.regressor = regressor

    def reset(self):
        self.dc_count = 0
        self.detector = None
        
    
    def step(self, price, time):
        if self.detector == None:
            self.detector = dclassifier.DC_EVENT_DETECTOR(self.theta, time, price)
            return [-1, 0]
        
        self.detector.step(price, time)

        #print(len(self.classified_data) , self.detector.get_dc_count())
        
        # if no new dc_event happend => return 0
        if self.dc_count == self.detector.get_dc_count():
            return [-1, 0]

        # if new dc_event happend
        # Will be replaced by the classification algortihm
        if self.detector.get_dc_count() == len(self.classified_data) or (self.detector.get_dc_count() - 1) == len(self.classified_data):
            return [-1, 0]
        
        if not self.classified_data.iloc[self.detector.get_dc_count()]["prev_os"]:
            self.dc_count += 1
            return [-1, 0]
        #
        
        dc_data = self.detector.get_dc()[-1]
        if dc_data['event_price'] >= 0:
            os_length = regression.eval_tree(self.regressor[0], dc_data['event_time'])
        #os_end = self.classified_data.iloc[self.detector.get_dc_count()]["start"]
        #os_start = self.classified_data.iloc[self.detector.get_dc_count() - 1]["end"]
        else:
            os_length = regression.eval_tree(self.regressor[1], dc_data['event_time'])

        self.dc_count += 1
        
        
        #return [os_end-os_start, 'Up' if self.classified_data.iloc[self.detector.get_dc_count() - 1]['event_price'] > 0 else 'Down']
        
        return [os_length, 'Up' if dc_data['event_price'] >= 0 else 'Down']

In [7]:
handlers = []
currency = "USDJPY"
for el in thresh_USDJPY:
    events, dcclass, classs = dclassifier.classify_split_timeseries(prices[currency], el)
    handlers.append(e_handler(events, el, UJ[el]))

In [8]:
backtesting_env = backtest.Optimize_eps(prices[currency], thresh_USDJPY, handlers)

In [9]:
l = backtesting_env.evaluate([1, 0, 0, 0, 0])

/home/linus/ghq/github.com/linux-03/MTDC/backtest.py:29: RuntimeWarning: invalid value encountered in scalar divide
  W = np.dot(weights, rev_points)/np.sum(weights)


vol 0.0 sharpe -inf ann_ret: 0.0 return 0.0 capital 100000


/home/linus/ghq/github.com/linux-03/MTDC/backtest.py:163: RuntimeWarning: divide by zero encountered in scalar divide
  return (ret - self.rf_rate)/vol


In [10]:
backtesting_env.hist

0       100000
1       100000
2       100000
3       100000
4       100000
         ...  
8251    100000
8252    100000
8253    100000
8254    100000
8255    100000
Length: 8256, dtype: int64